In [6]:
# pip install llama-index-core llama-index-llms-openai python-dotenv

In [7]:
from dotenv import load_dotenv

load_dotenv()

from llama_index.llms.openai import OpenAI
from llama_index.core.agent.workflow import FunctionAgent

In [8]:
def multiply(a: float,b:float)->float:
    return a*b

In [9]:
def add(a:float,b:float)->float:
    return a+b

In [10]:
llm = OpenAI(model="gpt-4o-mini")

In [11]:
# workflow = FunctionAgent(
#     tools=[multiply,add],
#     llm=llm,
#     system_prompt="you are an agent that can perform basic maths"
# )

In [12]:
# response = await workflow.run(user_msg="what is 20+(2*4)")
# print(response)

In [13]:
# pip install llama-index-tools-yahoo-finance

In [14]:
from llama_index.tools.yahoo_finance import YahooFinanceToolSpec

In [15]:
finance_tools = YahooFinanceToolSpec().to_tool_list

In [16]:
workflow = FunctionAgent(
    name="Agent",
    description="Useful for performing financial operations.",
    llm=OpenAI(model="gpt-4o-mini"),
    tools=finance_tools(),
    system_prompt="You are a helpful assistant.",
)

In [17]:
response = await workflow.run(
        user_msg="What's the current stock price of NVIDIA?"
    )
print(response)

The current stock price of NVIDIA (NVDA) is **$153.30**.


In [18]:
from llama_index.core.agent.workflow import AgentWorkflow

In [19]:
# pip install tavily-python

In [22]:
from tavily import AsyncTavilyClient
from llama_index.core.workflow import Context


async def search_web(query: str) -> str:
    """Useful for using the web to answer questions."""
    client = AsyncTavilyClient(api_key="tvly_api")
    return str(await client.search(query))


async def record_notes(ctx: Context, notes: str, notes_title: str) -> str:
    """Useful for recording notes on a given topic. Your input should be notes with a title to save the notes under."""
    current_state = await ctx.store.get("state")
    if "research_notes" not in current_state:
        current_state["research_notes"] = {}
    current_state["research_notes"][notes_title] = notes
    await ctx.store.set("state", current_state)
    return "Notes recorded."


async def write_report(ctx: Context, report_content: str) -> str:
    """Useful for writing a report on a given topic. Your input should be a markdown formatted report."""
    current_state = await ctx.store.get("state")
    current_state["report_content"] = report_content
    await ctx.store.set("state", current_state)
    return "Report written."


async def review_report(ctx: Context, review: str) -> str:
    """Useful for reviewing a report and providing feedback. Your input should be a review of the report."""
    current_state = await ctx.store.get("state")
    current_state["review"] = review
    await ctx.store.set("state", current_state)
    return "Report reviewed."

In [23]:
research_agent = FunctionAgent(
    name="ResearchAgent",
    description="search the web and record notes",
    system_prompt="you are a researcher .. hand off to WriteAgent when ready",
    llm = llm,
    tools=[search_web , record_notes],
    can_handoff_to=["WriteAgent"],
)

In [24]:
write_agent = FunctionAgent(
    name="WriteAgent",
    description="Writes a markdown report from the notes.",
    system_prompt="You are a writer… ask ReviewAgent for feedback when done.",
    llm=llm,
    tools=[write_report],
    can_handoff_to=["ReviewAgent", "ResearchAgent"],
)


In [25]:
review_agent = FunctionAgent(
    name="ReviewAgent",
    description="Reviews a report and gives feedback.",
    system_prompt="You are a reviewer…",  # etc.
    llm=llm,
    tools=[review_report],
    can_handoff_to=["WriteAgent"],
)

In [27]:
agent_workflow = AgentWorkflow(
    agents=[research_agent, write_agent, review_agent],
    root_agent=research_agent.name,
    initial_state={
        "research_notes": {},
        "report_content": "Not written yet.",
        "review": "Review required.",
    },
)

resp = await agent_workflow.run(
    user_msg="Write me a report on the history of the web …"
)
print(resp)

The report on the history of the web has been successfully created. Here’s a summary of the feedback for improvement:

1. **Structure and Overview**: The report is well-structured and provides a comprehensive overview of key milestones in the history of the web.

2. **Content Suggestions**:
   - Include specific examples of influential websites or technologies that shaped the web's development.
   - Add a brief discussion on the impact of the web on society and culture to enhance the report's depth.

If you would like to make any changes or need further assistance, please let me know!
